In [1]:
import pandas as pd
import numpy as np

In [2]:
loc= 'C:\\Users\\Ramshankar\\OneDrive - iLink Systems Inc\\Documents\\Costco\\Dataset\\Final data\\'

#Loading Date to week id map

data= pd.read_excel(loc + 'Inventory_final.xlsx', index_col=False)
data.head()

,date,product_identifier,department_identifier,category_of_product,outlet,state,sales,week_id,sell_price,year_month,net Sales,Temperature,Holiday
0,2020-01-01,74,11,others,111,Maharashtra,0,49,2.94,2012-01,0.00,26,0
1,2020-01-01,337,11,others,111,Maharashtra,1,49,7.44,2012-01,7.44,26,0
2,2020-01-01,423,12,others,111,Maharashtra,0,49,0.97,2012-01,0.00,26,0
3,2020-01-01,432,12,others,111,Maharashtra,0,49,4.97,2012-01,0.00,26,0
4,2020-01-01,581,21,fast_moving_consumer_goods,111,Maharashtra,0,49,4.88,2012-01,0.00,26,0


In [3]:
data= data.rename(columns={'product_identifier': 'Product_ID','department_identifier': 'Sub_Category',\
                           'category_of_product': 'Product_Category', 'outlet': 'Store', 'net Sales': 'Total_Revenue', \
                          'sales': 'Units_Sold', 'sell_price': 'Unit_Price'})

In [4]:
data= data[['date', 'Product_ID', 'Sub_Category', 'Product_Category', 'Store', 'Total_Revenue', 'Temperature', 'Holiday']]
data.head()

,date,Product_ID,Sub_Category,Product_Category,Store,Total_Revenue,Temperature,Holiday
0,2020-01-01,74,11,others,111,0.00,26,0
1,2020-01-01,337,11,others,111,7.44,26,0
2,2020-01-01,423,12,others,111,0.00,26,0
3,2020-01-01,432,12,others,111,0.00,26,0
4,2020-01-01,581,21,fast_moving_consumer_goods,111,0.00,26,0


In [5]:
data['ItemClass']= data['Product_Category'].astype(str) + "|" + data['Sub_Category'].astype(str) + "|" + data['Product_ID'].astype(str) 
data

,date,Product_ID,Sub_Category,Product_Category,Store,Total_Revenue,Temperature,Holiday,ItemClass
0,2020-01-01,74,11,others,111,0.00,26,0,others|11|74
1,2020-01-01,337,11,others,111,7.44,26,0,others|11|337
2,2020-01-01,423,12,others,111,0.00,26,0,others|12|423
3,2020-01-01,432,12,others,111,0.00,26,0,others|12|432
4,2020-01-01,581,21,fast_moving_consumer_goods,111,0.00,26,0,fast_moving_consumer_goods|21|581
...,...,...,...,...,...,...,...,...,...
157995,2022-02-28,2932,33,drinks_and_food,114,11.12,26,0,drinks_and_food|33|2932
157996,2022-02-28,2935,33,drinks_and_food,114,0.00,26,0,drinks_and_food|33|2935
157997,2022-02-28,3004,33,drinks_and_food,114,0.00,26,0,drinks_and_food|33|3004
157998,2022-02-28,3008,33,drinks_and_food,114,3.96,26,0,drinks_and_food|33|3008


In [7]:
data['ItemClass'].nunique()

50

In [63]:
data2 = data.reindex(columns= ['date', 'ItemClass', 'Store', 'Temperature', 'Holiday', 'Total_Revenue'])

In [64]:
data2['date'].min(), data2['date'].max()

(Timestamp('2020-01-01 00:00:00'), Timestamp('2022-02-28 00:00:00'))

In [65]:
data2_daily_sales = data2.groupby(['date', 'ItemClass', 'Store', 'Holiday'], as_index=False).agg(Avg_Temperature= ('Temperature', 'mean'), Avg_Sales = ('Total_Revenue', 'mean'))

In [79]:
data2_daily_sales.sort_values('date', inplace = True)

In [84]:
data2_daily_sales

,date,ItemClass,Store,Holiday,Avg_Temperature,Avg_Sales
0,2020-01-01,drinks_and_food|31|1629,111,0,26.0,8.82
127,2020-01-01,fast_moving_consumer_goods|22|1190,114,0,26.0,0.00
128,2020-01-01,fast_moving_consumer_goods|22|1196,111,0,26.0,0.00
129,2020-01-01,fast_moving_consumer_goods|22|1196,112,0,26.0,0.00
130,2020-01-01,fast_moving_consumer_goods|22|1196,113,0,26.0,0.00
...,...,...,...,...,...,...
157869,2022-02-28,fast_moving_consumer_goods|21|1054,112,0,26.0,19.68
157870,2022-02-28,fast_moving_consumer_goods|21|1054,113,0,26.0,26.24
157871,2022-02-28,fast_moving_consumer_goods|21|1054,114,0,26.0,3.28
157861,2022-02-28,drinks_and_food|33|3008,112,0,26.0,0.00


In [80]:
def series_to_supervised(data, window=1, lag=1, dropnan=True):
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    # Current timestep (t=0)
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    # Target timestep (t=lag)
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [81]:
'we are utilising the current timestep and the last 29 days, to forecast 90 days into the future.'

window = 29
lag = 15
data_series = series_to_supervised(data2_daily_sales.drop('date', axis=1), window=window, lag=lag)
data_series.head()

,ItemClass(t-29),Store(t-29),Holiday(t-29),Avg_Temperature(t-29),Avg_Sales(t-29),ItemClass(t-28),Store(t-28),Holiday(t-28),Avg_Temperature(t-28),Avg_Sales(t-28),...,ItemClass(t),Store(t),Holiday(t),Avg_Temperature(t),Avg_Sales(t),ItemClass(t+15),Store(t+15),Holiday(t+15),Avg_Temperature(t+15),Avg_Sales(t+15)
105,drinks_and_food|31|1629,111.0,0.0,26.0,8.82,fast_moving_consumer_goods|22|1190,114.0,0.0,26.0,0.0,...,fast_moving_consumer_goods|21|926,112,0,26.0,0.00,fast_moving_consumer_goods|22|1173,111.0,0.0,26.0,0.00
106,fast_moving_consumer_goods|22|1190,114.0,0.0,26.0,0.00,fast_moving_consumer_goods|22|1196,111.0,0.0,26.0,0.0,...,fast_moving_consumer_goods|21|926,113,0,26.0,0.00,fast_moving_consumer_goods|22|1173,112.0,0.0,26.0,0.00
107,fast_moving_consumer_goods|22|1196,111.0,0.0,26.0,0.00,fast_moving_consumer_goods|22|1196,112.0,0.0,26.0,0.0,...,fast_moving_consumer_goods|21|926,114,0,26.0,0.00,fast_moving_consumer_goods|22|1173,113.0,0.0,26.0,26.91
108,fast_moving_consumer_goods|22|1196,112.0,0.0,26.0,0.00,fast_moving_consumer_goods|22|1196,113.0,0.0,26.0,0.0,...,fast_moving_consumer_goods|21|972,111,0,26.0,0.00,fast_moving_consumer_goods|22|1190,111.0,0.0,26.0,0.00
109,fast_moving_consumer_goods|22|1196,113.0,0.0,26.0,0.00,fast_moving_consumer_goods|22|1196,114.0,0.0,26.0,0.0,...,fast_moving_consumer_goods|21|972,112,0,26.0,3.28,fast_moving_consumer_goods|22|1322,113.0,0.0,26.0,13.94


In [37]:
#data_series.to_csv('series.csv')

In [95]:
data_series.head(5)

,ItemClass(t-29),Store(t-29),Holiday(t-29),Avg_Temperature(t-29),Avg_Sales(t-29),ItemClass(t-28),Store(t-28),Holiday(t-28),Avg_Temperature(t-28),Avg_Sales(t-28),...,ItemClass(t),Store(t),Holiday(t),Avg_Temperature(t),Avg_Sales(t),ItemClass(t+15),Store(t+15),Holiday(t+15),Avg_Temperature(t+15),Avg_Sales(t+15)
105,drinks_and_food|31|1629,111.0,0.0,26.0,8.82,fast_moving_consumer_goods|22|1190,114.0,0.0,26.0,0.0,...,fast_moving_consumer_goods|21|926,112,0,26.0,0.00,fast_moving_consumer_goods|22|1173,111.0,0.0,26.0,0.00
106,fast_moving_consumer_goods|22|1190,114.0,0.0,26.0,0.00,fast_moving_consumer_goods|22|1196,111.0,0.0,26.0,0.0,...,fast_moving_consumer_goods|21|926,113,0,26.0,0.00,fast_moving_consumer_goods|22|1173,112.0,0.0,26.0,0.00
107,fast_moving_consumer_goods|22|1196,111.0,0.0,26.0,0.00,fast_moving_consumer_goods|22|1196,112.0,0.0,26.0,0.0,...,fast_moving_consumer_goods|21|926,114,0,26.0,0.00,fast_moving_consumer_goods|22|1173,113.0,0.0,26.0,26.91
108,fast_moving_consumer_goods|22|1196,112.0,0.0,26.0,0.00,fast_moving_consumer_goods|22|1196,113.0,0.0,26.0,0.0,...,fast_moving_consumer_goods|21|972,111,0,26.0,0.00,fast_moving_consumer_goods|22|1190,111.0,0.0,26.0,0.00
109,fast_moving_consumer_goods|22|1196,113.0,0.0,26.0,0.00,fast_moving_consumer_goods|22|1196,114.0,0.0,26.0,0.0,...,fast_moving_consumer_goods|21|972,112,0,26.0,3.28,fast_moving_consumer_goods|22|1322,113.0,0.0,26.0,13.94


In [69]:
#In this step, we have dropped any item values or store values which are different from the shifted columns.

last_ItemClass = 'ItemClass(t-%d)' % window
last_store = 'Store(t-%d)' % window

In [91]:
data_series[last_store].head(5)

105    111.0
106    114.0
107    111.0
108    112.0
109    113.0
Name: Store(t-29), dtype: float64

In [94]:
data_series['Store(t)'].head(5)

105    112
106    113
107    114
108    111
109    112
Name: Store(t), dtype: int64

In [89]:
#last_ItemClass = 'last_ItemClass(t-%d)' % window
(data_series['Store(t)'] == data_series[last_store]) == 'True'

105       False
106       False
107       False
108       False
109       False
          ...  
157854    False
157855    False
157856    False
157857    False
157858    False
Length: 157956, dtype: bool

In [70]:
data_series[last_ItemClass]

29                   drinks_and_food|31|1629
30                   drinks_and_food|31|1629
31                   drinks_and_food|31|1629
32                   drinks_and_food|31|1629
33                   drinks_and_food|31|1672
                         ...                
157980    fast_moving_consumer_goods|22|1322
157981    fast_moving_consumer_goods|22|1328
157982    fast_moving_consumer_goods|22|1328
157983    fast_moving_consumer_goods|22|1328
157984    fast_moving_consumer_goods|22|1328
Name: ItemClass(t-29), Length: 157956, dtype: object

In [78]:
2 == 2.0

True

In [ ]:
last_item = 'item(t-%d)' % window
last_store = 'store(t-%d)' % window
series = series[(series['store(t)'] == series[last_store])]
series = series[(series['item(t)'] == series[last_item])]